### U-Net implementation for the MapAI dataset

This is my own step-by-step implementation of U-Net

In [1]:
# !pip install --upgrade pip
# !conda install ipykernel
# !ipython kernel install --user --name=tf

In [2]:
# libraries

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # ignore tensorflow warnings

import tensorflow as tf

import numpy as np
import sys
import random

from tqdm import tqdm

from skimage.io import imread, imshow
from skimage.transform import resize

import matplotlib.pyplot as plt

# Define random seed
seed = 42
np.random.seed = seed


### Part 1: Prepare dataset

In [3]:
# 1) Define image dimensions and specify data location

img_width = 500
img_height = 500
img_channels = 3

# path to mapai dataset
train_path = '/home/shymon/data/mapai/train'
val_path = '/home/shymon/data/mapai/validation'

# Get the list of the subfolders for train/val
train_ids = next(os.walk(train_path))[1] 
val_ids = next(os.walk(val_path))[1]

train_ids, val_ids

print('Train folders: ', train_ids)
print('Validation folders: ', val_ids)

Train folders:  ['images', 'lidar', 'masks']
Validation folders:  ['images', 'lidar', 'masks']


In [4]:
# Show some images

from pathlib import Path
from PIL import Image
import numpy as np, pandas as  pd
from ipywidgets import interact, interactive, IntSlider, Select, RadioButtons, fixed, BoundedIntText

#### Plots ####

def plot_image_and_masks_from_df(imgidx, df, figsize=6, with_segm=True):
    imgfn = str(df.iloc[imgidx]['image'])
    if with_segm:
        maskfn = df.iloc[imgidx]['mask']
    f, ax = plt.subplots(figsize=(figsize,figsize))
    img = Image.open(imgfn)
    ax.imshow(img)
    if with_segm:
        mask = Image.open(maskfn)
        ax.imshow(mask, alpha=0.3)
    imgid = imgfn.split("/")[-1].split(".")[0]
    imgsz = img.size
    ax.set_title(f'{imgid}, {str(imgsz)}')
    ax.set_axis_off()
    return ax


def plot_image_lidar_and_masks_from_df(imgidx, df, figsize=6, with_segm=True):
    imgfn = str(df.iloc[imgidx]['image'])
    lidarfn = str(df.iloc[imgidx]['lidar'])
    if with_segm:
        maskfn = df.iloc[imgidx]['mask']
    f, ax = plt.subplots(1,2, figsize=(figsize,figsize))
    img = Image.open(imgfn)
    lidar = np.array(Image.open(lidarfn))
    ax[0].imshow(img)
    ax[1].imshow(img)
    if with_segm:
        mask = Image.open(maskfn)
        ax[0].imshow(mask, alpha=0.3)
        ax[1].imshow(lidar, alpha=0.3)
    imgid = imgfn.split("/")[-1].split(".")[0]
    imgsz = img.size
    ax[0].set_title(f'{imgid}, {str(imgsz)}')
    ax[0].set_axis_off()
    return ax

In [5]:
# Path to pandas file with labels

from pathlib import Path

# Update this to wherever you want to store data
DATADIR = Path("/home/shymon/data/mapai") 

csv_path = "/home/shymon/Documents/phd/03_CODE/LAB-Net/csv/train_val_original-2023-01-20.csv"
df = pd.read_csv(csv_path)

def update_paths(path_str):
    return path_str.replace("../../data", str(DATADIR))

df['image'] = df['image'].apply(update_paths)
df['lidar'] = df['lidar'].apply(update_paths)
df['mask'] = df['mask'].apply(update_paths)

df.head()

,image,lidar,mask,is_val,mask_percentage,is_building
0,/home/shymon/data/mapai/train/images/6179_495_...,/home/shymon/data/mapai/train/lidar/6179_495_4...,/home/shymon/data/mapai/train/masks/6179_495_4...,False,0.155224,True
1,/home/shymon/data/mapai/train/images/6051_690_...,/home/shymon/data/mapai/train/lidar/6051_690_8...,/home/shymon/data/mapai/train/masks/6051_690_8...,False,0.000000,False
2,/home/shymon/data/mapai/train/images/6121_865_...,/home/shymon/data/mapai/train/lidar/6121_865_5...,/home/shymon/data/mapai/train/masks/6121_865_5...,False,0.017824,True
3,/home/shymon/data/mapai/train/images/6173_630_...,/home/shymon/data/mapai/train/lidar/6173_630_2...,/home/shymon/data/mapai/train/masks/6173_630_2...,False,0.000000,False
4,/home/shymon/data/mapai/train/images/6147_481_...,/home/shymon/data/mapai/train/lidar/6147_481_4...,/home/shymon/data/mapai/train/masks/6147_481_4...,False,0.182664,True


In [6]:
# Interactive plot
interactive_plot = interactive(plot_image_and_masks_from_df, df=fixed(df),
                            imgidx = BoundedIntText(min=0, max=len(df)-1, step=1, value=0),
                               figsize = BoundedIntText(min=4, max=12, step=1, value=4),
                               with_segm= RadioButtons(options=[True,False], value=True, 
                                                      description="With segmentation"))

output = interactive_plot.children[-1]

# if i does not work, restart kernel
interactive_plot

interactive(children=(BoundedIntText(value=0, description='imgidx', max=8499), BoundedIntText(value=4, descrip…

### Part 2: Resizing training images

1. Resizing images is a critical preprocessing step in computer vision. Machine learning models tran faster on smaller images.
2. Images require normalization too. We need to divide the image matrix by dividing the pixel values by 256.

In [7]:
# Read all the images and resize them
X_train = np.zeros((len(train_ids), img_height, img_width, img_channels), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), img_height, img_width, 1), dtype=np.bool_)

In [8]:
# first we will try without resizing


### Part 3: Load and split data

In [9]:
# 2) Start building UNet model
# https://github.com/EhabR98/Image_segmentation_Unet-Tutorial

import tensorflow as tf
import numpy as np

# from test_utils import summary, comparator # import error


Convert images to tensors and apply normalization

In [10]:
df['image'] #, df['lidar'], df['mask']

img_list_ds = tf.data.Dataset.list_files(df['image'], shuffle = False)
mask_list_ds = tf.data.Dataset.list_files(df['mask'], shuffle = False)

images_fnames = tf.constant(df['image'])
masks_fnames = tf.constant(df['mask'])

print('# of images and masks: ', len(images_fnames), len(masks_fnames)) 

# of images and masks:  8500 8500


In [11]:
dataset = tf.data.Dataset.from_tensor_slices((images_fnames, masks_fnames))

for image, mask in dataset.take(1):
    print(image)
    print(mask)


tf.Tensor(b'/home/shymon/data/mapai/train/images/6179_495_44.tif', shape=(), dtype=string)
tf.Tensor(b'/home/shymon/data/mapai/train/masks/6179_495_44.tif', shape=(), dtype=string)


#### Part 4: Preprocess the data

Does resizing an image lead to loss of image quality?
Yes, resizing images can lead to loss of image quality. This can happen because when you resize an image, you are essentially adding or removing pixels. If you are increasing the size of an image, you need to add pixels, which can cause the image to look blurry or pixelated. If you are decreasing the size of an image, you need to remove pixels, which can cause important details to be lost. The extent of the loss of quality depends on various factors such as the resizing algorithm used, the size of the change in the image dimensions, and the complexity of the image content. To minimize loss of image quality when resizing, you can use resizing algorithms that preserve the edges and details in the image, such as bicubic interpolation or Lanczos resampling.

In [14]:
# Functions to decode images

def process_path(image_path, mask_path):

    # image
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=3) # for png -> works
    img = tf.image.convert_image_dtype(img, tf.float32)

    # mask
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=3)
    mask = tf.math.reduce_max(mask, axis=-1, keepdims=True)
    return img, mask

# Question does U-Net need to have 128x128 input image size? No, images can be any size.
# Question do we need to resize images? Images need be resized to a shape that can be divided by 32.
# In our case this would be 480 or 512 to upsize, since input is 500 x 500.

def preprocess(image, mask):
    input_image = tf.image.resize(image, (480, 480), method='bicubic')
    input_mask = tf.image.resize(mask, (480, 480), method='bicubic')

    # normalization
    input_image = input_image / 255.

    return input_image, input_mask

image_ds = dataset.map(process_path)
process_image_ds = image_ds.map(preprocess)

process_image_ds

<MapDataset element_spec=(TensorSpec(shape=(480, 480, 3), dtype=tf.float32, name=None), TensorSpec(shape=(480, 480, 1), dtype=tf.float32, name=None))>

### Part 5: U-Net

U-Net, named for its U-shape, was originally created in 2015 for tumor detection, but in the years since has become a very popular choice for other semantic segmentation tasks.

U-Net builds on a previous architecture called the Fully Convolutional Network, or FCN, which replaces the dense layers found in a typical CNN with a transposed convolution layer that upsamples the feature map back to the size of the original input image, while preserving the spatial information. This is necessary because the dense layers destroy spatial information (the "where" of the image), which is an essential part of image segmentation tasks. An added bonus of using transpose convolutions is that the input size no longer needs to be fixed, as it does when dense layers are used.

Unfortunately, the final feature layer of the FCN suffers from information loss due to downsampling too much. It then becomes difficult to upsample after so much information has been lost, causing an output that looks rough.

In [ ]:
# UNet implementation
# https://github.com/bnsreenu/python_for_image_processing_APEER/blob/master/tutorial118_binary_semantic_segmentation_using_unet.ipynb

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import concatenate

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate


def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)   #Not in the original network. 
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)  #Not in the original network
    x = Activation("relu")(x)

    return x

#Encoder block: Conv block followed by maxpooling

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p   

#Decoder block
#skip features gets input from encoder for concatenation

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

#Build Unet using the blocks
def build_unet(input_shape, n_classes):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024) #Bridge

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    if n_classes == 1:  #Binary
      activation = 'sigmoid'
    else:
      activation = 'softmax'

    outputs = Conv2D(n_classes, 1, padding="same", activation=activation)(d4)  #Change the activation based on n_classes
    print(activation)

    model = Model(inputs, outputs, name="U-Net")
    return model

In [ ]:
# Define input shape

img_height, img_width, img_channels = (480, 480, 3)

input_shape = (img_height, img_width, img_channels)
img_height, img_width, img_channels

In [ ]:
# find out which image size do we need as input
# Answer: 480

for i in range(0,512):
    #print(i)
    if i % 32 == 0:
        print(i)


In [ ]:
# Compile U-Net
model = build_unet(input_shape, n_classes=1)
model.compile(optimizer=Adam(learning_rate = 1e-3), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 124, 124, 256), (None, 125, 125, 256)]
# https://github.com/qubvel/segmentation_models/issues/1
# Solution: Height and width of input images should be divisible by 32 for all models

In [ ]:
# Loss function
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

In [ ]:
# Plot model

# !pip install pydot
# !pip install graphviz
# sudo apt install graphviz

# Plot model
tf.keras.utils.plot_model(model, "model.png", show_shapes=False, show_dtype=False, show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96)

In [ ]:
# Train model
# Requires tif file format

EPOCHS = 5
VAL_SUBSPLITS = 5
BUFFER_SIZE = 850 # https://stackoverflow.com/questions/64372390/what-does-buffer-size-do-in-tensorflow-dataset-shuffling
BATCH_SIZE = 32
process_image_ds.batch(BATCH_SIZE)

train_dataset = process_image_ds.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
print(process_image_ds.element_spec)

model_history = model.fit(train_dataset, epochs=EPOCHS)